# Extraction algorithm comparison

We create a synthetic 2D image, upon which we compare the results of `specreduce`'s `BoxcarExtract` and `HorneExtract` algorithms.

Since we control amplitude/uncertainty/etc., we can check that the results match our expectations. Among other things, we expect the Horne extraction's signal-to-noise ratio to outperform the boxcar's when using the whole scene as the aperture.

## Import packages

In [ ]:
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from astropy.modeling import models
from astropy.nddata import CCDData, VarianceUncertainty
from specreduce.extract import BoxcarExtract, HorneExtract
from specreduce.tracing import FlatTrace

In [ ]:
mpl.rcParams.update({'axes.titlesize': 18, 'axes.labelsize': 12,
                     'legend.fontsize': 12,  'axes.grid': False,
                     'grid.alpha': .5, 'grid.color': 'k',
                     'axes.edgecolor': 'k'})
np.random.seed(7) # use same random values in different sessions

## Create a 2D image

The flux in each column will follow a Gaussian distribution that we set using `astropy`'s modeling functionality.

We also add normally distributed noise throughout the image to make the extraction more difficult.

In [ ]:
nrows = 50*4
ncols = 40*4
sigma_pix = 4
sigma_noise = 1

In [ ]:
col_model = models.Gaussian1D(amplitude=1, mean=nrows/2, stddev=sigma_pix)
noise = np.random.normal(scale=sigma_noise, size=(nrows, ncols))

In [ ]:
index_arr = np.broadcast_to(np.arange(nrows)[:, np.newaxis], (nrows, ncols))
index_arr

In [ ]:
img = col_model(index_arr) + noise

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
im1 = ax.imshow(img, cmap='magma', origin='lower', vmin=0, vmax=1)
ax.set_title('synthetic 2D image')
fig.colorbar(im1)

In addition to the image, we also create variance and mask images for `HorneExtract`. The former was defined above; we use an array of zeros for the latter since this image has no "bad" pixels.

In [ ]:
variance = np.tile(sigma_noise, img.shape)
mask = np.zeros_like(img)

## Create a trace

Here, we manually set the trace to the middle row of the 2D image's y-axis.

In [ ]:
trace = FlatTrace(img, nrows/2)

## Calculate the extractions

In [ ]:
bxc = BoxcarExtract(img, trace)
bxc_result1d_slice = bxc(width=14)
bxc_result1d_whole = bxc(width=nrows)

In [ ]:
hrn = HorneExtract(img, trace)
hrn_result1d_whole = hrn(variance=variance,
                         mask=mask, unit=u.DN) # whole image is aperture

Note that `HorneExtract` can also take an `NDData` or `CCDData` image object as an argument. These are convenient because they allow for compiling the image, the variance, a mask, and any units into a single object. 

Once that's created, the only other argument needed is the trace. See this example with `CCDData`:

In [ ]:
mask = np.zeros_like(img)
var_obj = VarianceUncertainty(variance)
img_obj = CCDData(img, uncertainty=var_obj, mask=mask, unit=u.DN)

hrn2 = HorneExtract(img_obj, trace)
hrn2_result1d_whole = hrn2()

The results are the same either way:

In [ ]:
np.array_equal(hrn_result1d_whole.flux, hrn2_result1d_whole.flux)

## Compare results
The whole-image extractions come out as expected, with the Horne-extracted 1D spectrum showing a noticeably better signal-to-noise ratio than its windowless boxcar equivalent.

In [ ]:
fig2, ax2 = plt.subplots(figsize=(10, 8))

ax2.plot(hrn_result1d_whole.flux, c='#1d1160', label='1D spectrum, Horne, whole')
ax2.plot(bxc_result1d_whole.flux, c='cadetblue',
         label='1D spectrum, boxcar, whole', alpha=.5)
ax2.plot(bxc_result1d_slice.flux, c='cadetblue', linestyle='--',
         label='1D spectrum, boxcar, slice')
ax2.axhline(sigma_pix * np.sqrt(2*np.pi), c='#d4bd8a', linestyle='--',
            label=r'target ($\sigma_{spatial}$ * $\sqrt{2\pi}$)')

ax2.legend(fontsize=12)#, loc=(1.05,.5))
ax2.set_title('extracted 1D spectra')

The boxcar extraction can produce a similar result when its aperture is sliced to remove edge pixels. (Of course, that comes with the cost of losing any information those pixels contained.)

In [ ]:
fig3, ax3 = plt.subplots(figsize=(10, 4))

ax3.plot(hrn_result1d_whole.flux / np.nanmax(hrn_result1d_whole.flux),
         c='#1d1160', label='1D spectrum, Horne, whole')
# ax3.plot(bxc_result1d_whole.flux / np.nanmax(bxc_result1d_whole.flux),
#          c='cadetblue', label='1D spectrum, boxcar, whole')
ax3.plot(bxc_result1d_slice.flux / np.nanmax(bxc_result1d_slice.flux),
         c='cadetblue', label='1D spectrum, boxcar, slice', linestyle='--')

ax3.legend(fontsize=12)
ax3.set_title('extracted 1D spectra, normalized')